In [13]:
from pathlib import Path
import numpy as np
from PIL import Image
from scipy.ndimage import sobel
import json
import csv
import glob
import os

In [14]:
def load_image_gray(path: str) -> np.ndarray:

    img = Image.open(path).convert("L")            # conversion en niveaux de gris (8-bit)
    I = np.asarray(img, dtype=np.float32) / 255.0  # normalisation dans [0,1]
    return I


In [15]:
def make_grid_indices(h: int, w: int, n_points_x: int, n_points_y: int, patch_half: int = 8):

    x_min, x_max = patch_half, w - patch_half - 1
    y_min, y_max = patch_half, h - patch_half - 1
    if x_min > x_max or y_min > y_max:
        raise ValueError("Image trop petite pour des patches 16x16 avec ces marges.")

    # points régulièrement espacés
    xs_line = np.linspace(x_min, x_max, n_points_x, dtype=np.int32)
    ys_line = np.linspace(y_min, y_max, n_points_y, dtype=np.int32)

    # grille puis aplatissement en 1D
    X, Y = np.meshgrid(xs_line, ys_line)  # Y: lignes, X: colonnes
    xs = X.reshape(-1)
    ys = Y.reshape(-1)

    assert xs.shape == ys.shape == (n_points_x * n_points_y,)
    return xs, ys



In [19]:
def hog_descriptors(I, xs, ys, cell_size=4, cells_per_patch=4, nbins=8):

    # Vérif image en niveaux de gris
    assert I.ndim == 2, "Image doit être en niveaux de gris"

    # Taille du demi-patch (ici 8)
    patch_half = (cell_size * cells_per_patch) // 2

    # Gradients globaux
    Gx = sobel(I, axis=1, mode="reflect")
    Gy = sobel(I, axis=0, mode="reflect")
    magnitude = np.hypot(Gx, Gy)
    angle = np.arctan2(Gy, Gx)  # entre -pi et pi

    # Bins de l'histogramme
    bin_edges = np.linspace(-np.pi, np.pi, nbins + 1)

    # Sortie
    n_points = len(xs)
    descriptors = np.zeros((n_points, cells_per_patch * cells_per_patch * nbins), dtype=np.float32)

    # Boucle sur chaque point
    for i, (x, y) in enumerate(zip(xs, ys)):
        # Patch centré sur (x,y)
        patch_ang = angle[y - patch_half:y + patch_half, x - patch_half:x + patch_half]
        patch_mag = magnitude[y - patch_half:y + patch_half, x - patch_half:x + patch_half]

        # Diviser patch en cellules et calculer histogrammes
        desc = []
        for cy in range(cells_per_patch):
            for cx in range(cells_per_patch):
                cell_ang = patch_ang[cy*cell_size:(cy+1)*cell_size, cx*cell_size:(cx+1)*cell_size]
                cell_mag = patch_mag[cy*cell_size:(cy+1)*cell_size, cx*cell_size:(cx+1)*cell_size]
                hist, _ = np.histogram(cell_ang.ravel(), bins=bin_edges, weights=cell_mag.ravel())
                desc.extend(hist)  # concaténer dans une liste

        descriptors[i] = np.array(desc)

    return descriptors

# --- Exemple rapide ---
I = load_image_gray("STL10/stl10_raw/train/airplane/airplane_train_000.png")
h, w = I.shape
xs, ys = make_grid_indices(h, w, n_points_x=12, n_points_y=12, patch_half=8)
D = hog_descriptors(I, xs, ys)
print(D.shape, D.dtype)

(144, 128) float32


In [20]:
def compute_hog_dataset_npz(in_dir, out_file,
                            n_points_x=12, n_points_y=12,
                            cell_size=4, cells_per_patch=4, nbins=8, patch_half=8):
    in_dir = Path(in_dir)
    # 1) liste d'images
    img_paths = []
    for ext in ("*.png", "*.jpg", "*.jpeg"):
        img_paths.extend(glob.glob(str(in_dir / ext)))
    img_paths = sorted(img_paths)
    print("Nombre d'images trouvées :", len(img_paths))
    if len(img_paths) == 0:
        raise FileNotFoundError(f"Aucune image dans {in_dir}")

    # 2) calcul HOG pour chaque image, stockage dans une liste
    all_descs = []
    for i, img_path in enumerate(img_paths, 1):
        I = load_image_gray(img_path)
        h, w = I.shape
        xs, ys = make_grid_indices(h, w, n_points_x, n_points_y, patch_half)
        D = hog_descriptors(I, xs, ys, cell_size, cells_per_patch, nbins)
        all_descs.append(D)
        if i % 50 == 0 or i == len(img_paths):
            print(f"{i}/{len(img_paths)} images traitées")

    # 3) empilement et sauvegarde .npz (compressé)
    ALL = np.vstack(all_descs).astype(np.float32)  # (N_total, 128)
    np.savez_compressed(out_file, all_descs=ALL)
    print("Sauvegardé :", out_file, "| shape =", ALL.shape)

In [21]:
# HOG pour toutes les images de airplane/train → un seul .npz
compute_hog_dataset_npz(
    in_dir="STL10/stl10_raw/train/airplane",
    out_file="HOG_computation/airplane_train.npz"
)


Nombre d'images trouvées : 500
50/500 images traitées
100/500 images traitées
150/500 images traitées
200/500 images traitées
250/500 images traitées
300/500 images traitées
350/500 images traitées
400/500 images traitées
450/500 images traitées
500/500 images traitées
Sauvegardé : HOG_computation/airplane_train.npz | shape = (72000, 128)


In [24]:
compute_hog_dataset_npz(
    in_dir="STL10/stl10_raw/train/bird",
    out_file="HOG_computation/bird_train.npz"
)

Nombre d'images trouvées : 500
50/500 images traitées
100/500 images traitées
150/500 images traitées
200/500 images traitées
250/500 images traitées
300/500 images traitées
350/500 images traitées
400/500 images traitées
450/500 images traitées
500/500 images traitées
Sauvegardé : HOG_computation/bird_train.npz | shape = (72000, 128)


In [25]:
compute_hog_dataset_npz(
    in_dir="STL10/stl10_raw/train/car",
    out_file="HOG_computation/car_train.npz"
)

Nombre d'images trouvées : 500
50/500 images traitées
100/500 images traitées
150/500 images traitées
200/500 images traitées
250/500 images traitées
300/500 images traitées
350/500 images traitées
400/500 images traitées
450/500 images traitées
500/500 images traitées
Sauvegardé : HOG_computation/car_train.npz | shape = (72000, 128)


In [26]:
compute_hog_dataset_npz(
    in_dir="STL10/stl10_raw/train/cat",
    out_file="HOG_computation/cat_train.npz"
)

Nombre d'images trouvées : 500
50/500 images traitées
100/500 images traitées
150/500 images traitées
200/500 images traitées
250/500 images traitées
300/500 images traitées
350/500 images traitées
400/500 images traitées
450/500 images traitées
500/500 images traitées
Sauvegardé : HOG_computation/cat_train.npz | shape = (72000, 128)


In [27]:
compute_hog_dataset_npz(
    in_dir="STL10/stl10_raw/train/deer",
    out_file="HOG_computation/deer_train.npz"
)

Nombre d'images trouvées : 500
50/500 images traitées
100/500 images traitées
150/500 images traitées
200/500 images traitées
250/500 images traitées
300/500 images traitées
350/500 images traitées
400/500 images traitées
450/500 images traitées
500/500 images traitées
Sauvegardé : HOG_computation/deer_train.npz | shape = (72000, 128)


In [28]:
compute_hog_dataset_npz(
    in_dir="STL10/stl10_raw/train/dog",
    out_file="HOG_computation/dog_train.npz"
)

Nombre d'images trouvées : 500
50/500 images traitées
100/500 images traitées
150/500 images traitées
200/500 images traitées
250/500 images traitées
300/500 images traitées
350/500 images traitées
400/500 images traitées
450/500 images traitées
500/500 images traitées
Sauvegardé : HOG_computation/dog_train.npz | shape = (72000, 128)


In [29]:
compute_hog_dataset_npz(
    in_dir="STL10/stl10_raw/train/horse",
    out_file="HOG_computation/horse_train.npz"
)

Nombre d'images trouvées : 500
50/500 images traitées
100/500 images traitées
150/500 images traitées
200/500 images traitées
250/500 images traitées
300/500 images traitées
350/500 images traitées
400/500 images traitées
450/500 images traitées
500/500 images traitées
Sauvegardé : HOG_computation/horse_train.npz | shape = (72000, 128)


In [30]:
compute_hog_dataset_npz(
    in_dir="STL10/stl10_raw/train/monkey",
    out_file="HOG_computation/monkey_train.npz"
)

Nombre d'images trouvées : 500
50/500 images traitées
100/500 images traitées
150/500 images traitées
200/500 images traitées
250/500 images traitées
300/500 images traitées
350/500 images traitées
400/500 images traitées
450/500 images traitées
500/500 images traitées
Sauvegardé : HOG_computation/monkey_train.npz | shape = (72000, 128)


In [31]:
compute_hog_dataset_npz(
    in_dir="STL10/stl10_raw/train/ship",
    out_file="HOG_computation/ship_train.npz"
)

Nombre d'images trouvées : 500
50/500 images traitées
100/500 images traitées
150/500 images traitées
200/500 images traitées
250/500 images traitées
300/500 images traitées
350/500 images traitées
400/500 images traitées
450/500 images traitées
500/500 images traitées
Sauvegardé : HOG_computation/ship_train.npz | shape = (72000, 128)


In [32]:
compute_hog_dataset_npz(
    in_dir="STL10/stl10_raw/train/truck",
    out_file="HOG_computation/truck_train.npz"
)

Nombre d'images trouvées : 500
50/500 images traitées
100/500 images traitées
150/500 images traitées
200/500 images traitées
250/500 images traitées
300/500 images traitées
350/500 images traitées
400/500 images traitées
450/500 images traitées
500/500 images traitées
Sauvegardé : HOG_computation/truck_train.npz | shape = (72000, 128)
